In [3]:
import requests
from bs4 import BeautifulSoup as bs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime


### Web scrape the food price index data

In [70]:
page = requests.get('https://ycharts.com/indicators/agriculture_index_world_bank')
page = page.content
soup = bs(page, 'html.parser')
dataTableBox = soup.find('div', {"id": "dataTableBox"})
col1 = []
col2 = []
# get the data from table into lists
for tr in dataTableBox.find_all('tr')[2:]:
    cols = tr.find_all('td')
    if(len(cols) == 0 ): # for empty div comming in between
        continue
#     print(datetime.strptime(cols[0].text, '%m. %d, %y'))
    col1.append(cols[0].text)
    col2.append(float(cols[1].text.strip()))

# Create dataframe from the lists of data
pd1_input_list = {'Month':col1,'Price_Index':col2}
df1 = pd.DataFrame(pd1_input_list)
# Write the dataframe data to excel file
writer = pd.ExcelWriter('monthly_price_index.xlsx', engine='xlsxwriter')
df1.to_excel(writer, sheet_name='Sheet1')
writer.save()

### Download the data from url into csv

In [169]:
url = 'https://www.ers.usda.gov/webdocs/DataFiles/54529/SandU%20Fresh.xlsx?v=43189'
r = requests.get(url)
# print(r.content)
with open("abc.csv",'wb') as f:
    # write the contents of the response (r.content)
    # to a new file in binary mode.
    f.write(r.content)


### Getting Data from API:

In [1]:
import requests
import sys
import json

key = 'GlanxFCOEPxiiEXJK37dPGQjnfM0wFfrZf2uuWti'
url = 'https://api.nal.usda.gov/ndb/nutrients/?format=json&api_key=GlanxFCOEPxiiEXJK37dPGQjnfM0wFfrZf2uuWti&nutrients=208&nutrients=203&nutrients=204&nutrients=205&nutrients=291&nutrients=269&nutrients=211&nutrients=212&fg=1100'
response = requests.get(url)
x = response.json()
# df2 = pd.read_json('https://api.nal.usda.gov/ndb/nutrients/?format=json&api_key=GlanxFCOEPxiiEXJK37dPGQjnfM0wFfrZf2uuWti&nutrients=208&nutrients=203&nutrients=204&nutrients=205&nutrients=291&nutrients=269&nutrients=211&nutrients=212&fg=1100')

In [4]:
# need to grab the nutrient information from nested dictionary structure
df3 = pd.io.json.json_normalize(x['report']['foods'])
df3.columns = df3.columns.map(lambda x: x.split(".")[-1])
# df3

In [146]:
df3['nutrients'][0] # to grab the nutrient information of first food item
df3['nutrients'][0][0]['value'] # gives glucose value for first food item

'0.03'

In [150]:
# vegetables with raw names
raw_names = [ item for item in df3['name'] if item.find('raw') > 0]
raw_names
# Vegetable list in food prices data
veg_prices = ['Artichokes','Asparagus','Broccoli','Brussels Sprouts','Cabbage','Carrots','Cauliflower','Celery','Collard Greens','Cucumbers','Eggplant','Escarole & Endive','Garlic','Head Lettuce','Leaf & Romain Lettuce','Kale','Mushrooms','Mustard greens','Okra','Onions','Pepper, Bell','Potatoes','Pumpkin','Radishes','Snap Beans','Spinach','Squash','Sweet Corn','Tomatoes','Turnip Greens']



['Alfalfa seeds, sprouted, raw',
 'Asparagus, raw',
 'Beans, snap, green, raw',
 'Broccoli raab, raw',
 'Broccoli, raw',
 'Brussels sprouts, raw',
 'Cabbage, raw',
 'Cabbage, red, raw',
 'Carrots, baby, raw',
 'Carrots, raw',
 'Cauliflower, raw',
 'Celery, raw',
 'Corn, sweet, yellow, raw',
 'Cucumber, peeled, raw',
 'Cucumber, with peel, raw',
 'Eggplant, raw',
 'Kale, raw',
 'Lettuce, butterhead (includes boston and bibb types), raw',
 'Lettuce, cos or romaine, raw',
 'Lettuce, green leaf, raw',
 'Lettuce, iceberg (includes crisphead types), raw',
 'Lettuce, red leaf, raw',
 'Mushroom, white, exposed to ultraviolet light, raw',
 'Mushrooms, Chanterelle, raw',
 'Mushrooms, enoki, raw',
 'Mushrooms, maitake, raw',
 'Mushrooms, morel, raw',
 'Mushrooms, oyster, raw',
 'Mushrooms, portabella, exposed to ultraviolet light, raw',
 'Mushrooms, portabella, raw',
 'Mushrooms, shiitake, raw',
 'Mushrooms, white, raw',
 'Okra, raw',
 'Onions, raw',
 'Onions, sweet, raw',
 'Peas, green, raw',
 '

In [5]:
# for i in my_data[:5]:
#     print(i['name'])
#     print(i['nutrients'])
my_data = x['report']['foods']
nut_info = [nut for veggie in my_data for nut in veggie['nutrients']]
veg = [veggie['name'] for veggie in my_data for nut in veggie['nutrients']]

In [206]:
nut_col = df3['nutrients']
for item in nut_col[:10]:
    print(item)

[{'nutrient_id': '211', 'nutrient': 'Glucose (dextrose)', 'unit': 'g', 'value': '0.03', 'gm': 0.08}, {'nutrient_id': '212', 'nutrient': 'Fructose', 'unit': 'g', 'value': '0.04', 'gm': 0.12}, {'nutrient_id': '203', 'nutrient': 'Protein', 'unit': 'g', 'value': '1.32', 'gm': 3.99}, {'nutrient_id': '269', 'nutrient': 'Sugars, total', 'unit': 'g', 'value': '0.07', 'gm': 0.2}, {'nutrient_id': '204', 'nutrient': 'Total lipid (fat)', 'unit': 'g', 'value': '0.23', 'gm': 0.69}, {'nutrient_id': '205', 'nutrient': 'Carbohydrate, by difference', 'unit': 'g', 'value': '0.69', 'gm': 2.1}, {'nutrient_id': '208', 'nutrient': 'Energy', 'unit': 'kcal', 'value': '8', 'gm': 23.0}, {'nutrient_id': '291', 'nutrient': 'Fiber, total dietary', 'unit': 'g', 'value': '0.6', 'gm': 1.9}]
[{'nutrient_id': '211', 'nutrient': 'Glucose (dextrose)', 'unit': 'g', 'value': '0.29', 'gm': 0.24}, {'nutrient_id': '212', 'nutrient': 'Fructose', 'unit': 'g', 'value': '0.02', 'gm': 0.02}, {'nutrient_id': '203', 'nutrient': 'Prot

In [207]:
final_map2 = {}
from collections import defaultdict
final_map2 = defaultdict(list)
for item in nut_col:
    final_map2['gm'].append(item['gm'])

TypeError: list indices must be integers or slices, not str

In [6]:
final_map = {}
from collections import defaultdict
final_map = defaultdict(list)
for item in nut_info:
    final_map['gm'].append(item['gm'])
# print(final_map)

In [7]:
nutrients_df_raw = pd.DataFrame(final_map)
# print(nutrients_df_raw.head(20))
# print (np.reshape(nutrients_df_raw.values,(750,8)))
nutrients_df_raw = pd.DataFrame(np.reshape(nutrients_df_raw.values,(150,8)), 
                    columns=['Glucose','Fructose','Protein','Sugars','fat','Carbohydrate' , 'Energy', 'Fiber'])
nutrients_df_raw['Vegetable'] = veg[0::8]


In [8]:
nutrients_df_raw

,Glucose,Fructose,Protein,Sugars,fat,Carbohydrate,Energy,Fiber,Vegetable
0,0.08,0.12,3.99,0.2,0.69,2.1,23,1.9,"Alfalfa seeds, sprouted, raw"
1,0.24,0.02,2.89,0.99,0.34,11.39,51,5.7,"Artichokes, (globe or french), cooked, boiled,..."
2,0.24,0.02,2.89,0.99,0.34,11.95,53,5.7,"Artichokes, (globe or french), cooked, boiled,..."
3,0.42,0.79,2.4,1.3,0.22,4.11,22,2,"Asparagus, cooked, boiled, drained"
4,0.42,0.79,2.4,1.3,0.22,4.11,22,2,"Asparagus, cooked, boiled, drained, with salt"
5,0.65,1,2.2,1.88,0.12,3.88,20,2.1,"Asparagus, raw"
6,0.29,0.33,1.12,0.78,0.46,4.32,22,1.9,"Beans, snap, green, canned, no salt added, dra..."
7,0.63,0.62,1.05,1.44,0.41,4.19,21,1.9,"Beans, snap, green, canned, regular pack, drai..."
8,0.83,1.36,1.98,2.6,0.41,6.98,33,3.4,"Beans, snap, green, frozen, all styles, microw..."
9,0.92,1.04,1.79,2.21,0.21,7.54,33,2.6,"Beans, snap, green, frozen, all styles, unprep..."


In [151]:
veg_nut_df = pd.DataFrame({'name': veg, 'nutrient':nut_info})
veg_nut_df

,name,nutrient
0,"Alfalfa seeds, sprouted, raw","{'nutrient_id': '211', 'nutrient': 'Glucose (d..."
1,"Alfalfa seeds, sprouted, raw","{'nutrient_id': '212', 'nutrient': 'Fructose',..."
2,"Alfalfa seeds, sprouted, raw","{'nutrient_id': '203', 'nutrient': 'Protein', ..."
3,"Alfalfa seeds, sprouted, raw","{'nutrient_id': '269', 'nutrient': 'Sugars, to..."
4,"Alfalfa seeds, sprouted, raw","{'nutrient_id': '204', 'nutrient': 'Total lipi..."
5,"Alfalfa seeds, sprouted, raw","{'nutrient_id': '205', 'nutrient': 'Carbohydra..."
6,"Alfalfa seeds, sprouted, raw","{'nutrient_id': '208', 'nutrient': 'Energy', '..."
7,"Alfalfa seeds, sprouted, raw","{'nutrient_id': '291', 'nutrient': 'Fiber, tot..."
8,"Artichokes, (globe or french), cooked, boiled,...","{'nutrient_id': '211', 'nutrient': 'Glucose (d..."
9,"Artichokes, (globe or french), cooked, boiled,...","{'nutrient_id': '212', 'nutrient': 'Fructose',..."


In [ ]:

https://api.nal.usda.gov/ndb/nutrients/?format=json&api_key=GlanxFCOEPxiiEXJK37dPGQjnfM0wFfrZf2uuWti&nutrients=208&nutrients=203&nutrients=204&nutrients=205&nutrients=291&nutrients=269&nutrients=211&nutrients=212&fg=1100

https://api.nal.usda.gov/ndb/nutrients/?format=json&api_key=DEMO_KEY&nutrients=208&nutrients=203&nutrients=204&nutrients=205&nutrients=291&nutrients=269&nutrients=211&nutrients=212&fg='Vegetables and Vegetable Products'